In [ ]:
import os
try:                  # get to root of project
    print(od)
except NameError:
    od = os.getcwd()
    
os.chdir(od + '/..')
print(os.getcwd())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams, plot, scatter

from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, Ticker, FixedTicker, Arrow, VeeHead

import numpy as np
import pandas as pd
import glob as glob
import bs4 as bs
import json
import re

import pubchempy as pc

from collections import Hashable
from hashlib import md5

import tqdm
from keras_tqdm import TQDMNotebookCallback

from keras import backend as Kb
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, Dropout, GaussianNoise
from keras.layers import Activation
from keras.layers import Conv1D, Conv2D, MaxPool1D, MaxPool2D, UpSampling1D, UpSampling2D
from keras.models import Model
from keras import regularizers, objectives

from sklearn import preprocessing, model_selection

SEED = 1337
np.random.seed(SEED)


In [ ]:
class Autoencoder(object):
    """
    Base class for all-purpose autoencoder. VAE, CNN-AE, etc will be built off of this.

    Input -> Encoder -> Z Latent Vector -> Decoder -> Output
    """
    def __init__(self,
                 input_shape=(28, 28, 1),
                 latent_dim=2,  # Size of the encoded vector
                 batch_size=100, # size of minibatch
                 compile_decoder=False # create the decoder. Not necessary for every use case
                 ):
        self.model = None
        self.encoder = None
        self.decoder = None
        self.batch_size = batch_size
        self.latent_dim = latent_dim
        self.compile_decoder = compile_decoder
        assert Kb.image_dim_ordering() == 'tf', 'Cannot support Theano ordering! Use TF ordering! #tensorflowmasterrace'

        # input image dimensions
        self.input_shape = input_shape
        # self.data_shape = input_shape[1:] # Shape of a single sample
        if len(input_shape) == 4:
            self.img_rows, self.img_cols, self.img_stacks, self.img_chns = input_shape
        elif len(input_shape) == 3:
            self.img_rows, self.img_cols, self.img_chns = input_shape
        elif len(input_shape) == 2:
            self.img_rows, self.img_cols = input_shape
            self.img_chns = 1
        elif len(input_shape) == 1:
            self.img_rows = input_shape[0]  # todo: test this
        else:
            raise ValueError("Invalid input shape: {}".format(input_shape))

    def rollup_decoder(self, z, z_input, layers_list):
        """
        Takes a list of Keras layers and returns the decoder back-half and the standalone decoder model
        :param z: Layer corresponding to the latent space vector
        :param z_input: Layer corresponding to the decoder input
        :param layers_list: List of layers to roll up
        :return:
        """
        ae = AE_Dec()
        dc = AE_Dec()
        last_ae = z
        last_dc = z_input
        for i, layer in enumerate(layers_list):
            #             if i ==0:
            try:
                last_ae = layer(last_ae)
            except ValueError:
                print(layers_list[i])
                raise
            print(last_ae, last_ae.shape)
            if self.compile_decoder:
                last_dc = layer(last_dc)
        return last_ae, last_dc


class AE_Dec(object):
    """
    Dummy object for reasons I can't remember. This may be deprecated.
    """
    def __init__(self):
        pass


class VAE(Autoencoder):
    """
    Variational Autoencoder.
    """
    def __init__(self,
                 input_shape=(28, 28, 1),
                 latent_dim=2,  # Size of the encoded vector
                 batch_size=100,  # size of minibatch
                 epsilon_std=1.0, # This is the stddev for our normal-dist sampling of the latent vector
                 compile_decoder=False
                 ):
        super().__init__(input_shape=input_shape, latent_dim=latent_dim, batch_size=batch_size,
                         compile_decoder=compile_decoder)
        # Necessary to instantiate this as instance variables such that they can be passed to the loss function (internally), since loss functions are
        # all of the form lossfn(y_true, y_pred)
        self.epsilon_std = epsilon_std
        self.z_mean = Dense(latent_dim)
        self.z_log_var = Dense(latent_dim)



    def sampling(self, args):
        """This is what makes the variational technique happen.
        """
        # Forging our latent vector from the reparameterized mean and std requires some sampling trickery
        # that admittedly I do not understand in the slightest at this point in time
        z_mean, z_log_var = args
        batch = Kb.shape(z_mean)[0]
        dim = Kb.int_shape(z_mean)[1]
        epsilon = Kb.random_normal(shape=(batch, dim),
                                          mean=0., stddev=self.epsilon_std)
        # We return z_mean + epsilon*sigma^2. Not sure why we use log var
        # Basically, create a random variable vector from the distribution
        # We are learning a distribution (mu, var) which represents the input
        return z_mean + Kb.exp(z_log_var) * epsilon

    def vae_loss(self, x, x_decoded_mean):
        """
        Custom loss function for VAE. Uses Kullback-Leibler divergence.

        Notes from fchollet: binary_crossentropy expects a shape (batch_size, dim) for x and x_decoded_mean,
        so we MUST flatten these!
        :param x:
        :param x_decoded_mean:
        :return:
        """

        x = Kb.flatten(x)
        x_decoded_mean = Kb.flatten(x_decoded_mean)
        shape_coef = np.product(self.input_shape)
        xent_loss = shape_coef * objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * Kb.mean(
            1 + self.z_log_var - Kb.square(self.z_mean) - Kb.exp(self.z_log_var), axis=-1)
        # Kullback–Leibler divergence. so many questions about this one single line
        return xent_loss + kl_loss


In [ ]:
class VAE_matembed(VAE):
    def __init__(self,
                 input_shape=(30, 6), latent_dim=2, n_classes=10,  batch_size=100,  
                 n_stacks=3,  # Number of convolayers to stack, this boosts performance of the network dramatically
                 intermediate_dim=10,  # Size of the dense layer after convs
                 n_filters=6,  # Number of filters in the first layer
                 px_conv=1,  # Default convolution window size
                 dropout_p=0.1,  # Default dropout rate
                 epsilon_std=1.0,  # This is the stddev for our normal-dist sampling of the latent vector
                 compile_decoder=True,
                 ):

        # This is my original crossfire network, and it works. As such, it has apprentice marks all over
        # Reconstructing as-is before tinkering
        # Based heavily on https://github.com/fchollet/keras/blob/master/examples/variational_autoencoder_deconv.py
        # and https://groups.google.com/forum/#!msg/keras-users/iBp3Ngxll3k/_GbY4nqNCQAJ

        super().__init__(input_shape=input_shape, latent_dim=latent_dim, batch_size=batch_size, epsilon_std=epsilon_std,
                         compile_decoder=compile_decoder)

        x_in = Input(batch_shape=(batch_size,) + self.input_shape, name='main_input')
        stack = Conv1D(n_filters, px_conv, activation='relu', name='stack1')(x_in)
        print(stack.shape)
        flat = Flatten()(stack)
        hidden_1 = Dense(intermediate_dim, activation='relu', name='intermezzo')(flat)
        
        z_mean = Dense(latent_dim)(hidden_1)
        z_log_var = Dense(latent_dim)(hidden_1)

        # Make these instance vars so X-Ent can use them. Probably a better way out there
        self.z_mean = z_mean
        self.z_log_var = z_log_var
        z = Lambda(self.sampling, output_shape=(latent_dim,), name='latent_z')([z_mean, z_log_var])
        
        decoder_hidden = Dense(intermediate_dim, activation='relu')
        print(input_shape)
        output_shape = (batch_size, input_shape[0], input_shape[1], n_filters)
        
        decoder_restack = Dense(input_shape[0] * input_shape[1], activation='relu')
        print(decoder_restack)
        decoder_reshape = Reshape(output_shape[1:3])
        decoder_mean_squash = Conv1D(n_filters, px_conv, activation='sigmoid')
        
        
        decoder_input = Input(shape=(latent_dim,))
        
        layers_list = [decoder_hidden, decoder_restack, decoder_reshape, decoder_mean_squash]
        
        ae, dc = self.rollup_decoder(z, decoder_input, layers_list)
        
        # Now we create the actual models. We also compile them automatically, this could be isolated later
        # Primary model - VAE
        self.model = Model(x_in, ae)
        self.model.compile(optimizer='rmsprop', loss=self.vae_loss)
         # build a model to project inputs on the latent space
        self.encoder = Model(x_in, self.z_mean)
        if self.compile_decoder:
            # reconstruct the output from latent space
            self.decoder = Model(decoder_input, dc)
            

In [ ]:
basedir = '/home/mike/data/chemistry/compatibility/'
files = glob.glob(basedir + 'parsed/' + '*.csv')
files

In [ ]:
frames = [pd.read_csv(fn) for fn in files[:2]]
for frame in frames:
    frame.columns = map(str.lower, frame.columns)

In [ ]:
n = 0
print(frames[n].shape)
frames[n].head()

In [ ]:
n = 1
print(frames[n].shape)
frames[n].head()

In [ ]:
ddict = {#'fluorocarbon (fkm)': 'fluoroelastomer (fkma)',
        '316 stainless steel': 'ss316', 'stainless steel - 316': 'ss316',
         '304 stainless steel': 'ss304', 'stainless steel - 304': 'ss304',
         'buna': 'buna', 'buna n (nitrile)': 'buna', 'buna-n (nitrile)': 'buna',
         'pvdf (kynar)': 'pvdf',
         'teflon': 'ptfe',
         'acetal (delrin)': 'acetal', 'polyacetal': 'acetal',
         'polyurethane': 'urethane',
         'cast/ductile iron': 'cast iron',
         'polyetherether ketone (peek)': 'peek',
         'hastelloy c': 'hastelloy-c',
         'epr, epdm': 'epdm',
         'santoprene (epdm & polypropylene)': 'santoprene',
         'polypropylene': 'pp'
        }

for frame in frames:
    frame.rename(columns=ddict, inplace=True)

In [ ]:
for frame in frames:
    frame.set_index('chemical', inplace=True)


In [ ]:
for frame in frames:
    frame.replace('-', np.nan, inplace=True)
    print(frame[frame.columns[0]].unique())

In [ ]:
def get_label_columns(mydf):
    labs = {'A': 4, 'B': 3, 'C': 2, 'D': 1, np.NaN: 0}
    lb = preprocessing.LabelBinarizer()
    lb.fit(mydf[mydf.columns[0]].replace(labs))
    cols = [col.replace(labs) for name, col in mydf.iteritems()]
    try:
        cols = [lb.transform(col) for col in cols]
    except ValueError:
        s = set()
        for name, col in mydf.iteritems():
            s.update(set(col.unique()))
        raise ValueError('Could not map labels. Found values: {}'.format(s))
    return np.stack(cols, axis=1)

In [ ]:
df = frames[0]
ary = get_label_columns(df)
ary.shape

In [ ]:
# ary[1]

#### now to create the data set
for every chemical C and material M, create a data pair such that the input tensor is nM x 6, the label data for C is filled with 0 for the input, 6th column is hot, and left for the output. Output should be of shape (nC * nM, nM, 6)
Also make a NaN mask for when the label to be predicted would be NaN (zeroth column)

In [ ]:
nchem, nmat, ncat = ary.shape # num_chemicals, num_materials, num_categories (incl NaN)
x_set = np.zeros((nchem*nmat, nmat, ncat+1))
y_set = np.copy(x_set)
print(x_set.shape)
blank_layer = np.zeros((nmat, 1)) 
for row in range(nchem):
    for col in range(nmat):
        idx = row*nmat + col
        insert = np.append(ary[row], np.copy(blank_layer), axis=1)
        y_set[idx] = np.copy(insert) # not sure if necessary but better safe than sorry
        insert[col] = 0
        insert[col, ncat] = 1
        x_set[idx] = insert
        

In [ ]:
n = 20
# x_set[n]

In [ ]:
# y_set[n]

In [ ]:
latent_dim = 5
input_shape = x_set.shape[1:]
batch_size = 100

In [ ]:
ae = VAE_matembed(input_shape=input_shape, latent_dim=latent_dim, batch_size=None,
                  compile_decoder=False)
vae = ae.model
print(vae.summary())

In [ ]:
def pad_batch_size(ary, batch_size=100):
    xs = batch_size - (len(ary) % batch_size)
    return np.append(ary, ary[:xs], axis=0)

def train_test_split_pad(*arrays, batch_size=100, test_size=None, random_state=None):
    datasets = model_selection.train_test_split(*arrays, test_size=test_size, random_state=random_state)
    return [pad_batch_size(ary, batch_size=batch_size) for ary in datasets]
        

x_train, x_test, y_train, y_test = train_test_split_pad(x_set, y_set, batch_size=batch_size, random_state=SEED)
print( x_train.shape, x_test.shape)

In [ ]:
assert 0, 'halt'

In [ ]:
vae.fit(x_train, x_train,
        shuffle=True,
        epochs=10,
        batch_size=batch_size,
        validation_data=(x_test, x_test),
        verbose=False,
        callbacks=[TQDMNotebookCallback()])

In [ ]:
n = 0
bs = 10
x_in = x_train[n: n+bs]
print(x_in.shape)
yp = vae.predict(x_in, bs)